In [186]:
import pandas as pd
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
import Levenshtein as Lv
import json
import nltk
import string
from heapq import nlargest

In [187]:
data = pd.read_csv('All_campaigns.csv',skiprows=2,low_memory=False)

In [188]:
data

,Account name,Customer ID,Campaign,Ad group,Campaign ID,Ad group ID,Keyword ID,Search keyword,Currency code,Clicks,...,Avg. CPC,Converted currency code,Avg. CPC (Converted currency),Cost,Cost (Converted currency),Conversions,View-through conv.,Cost / conv.,Cost / conv. (Converted currency),Conv. rate
0,HealthCare.com U65,476-322-7661,U65_HC_SL_G_RO_state-terms-kwurl-experiment_AD...,WV.1.1.10.1.5.00.1.0i.2_NF_NA_Health-Insurance...,10760603187,109125946314,668909987272,west virginia state insurance exchange plans,USD,0,...,--,--,--,0.0,--,0.0,0,--,--,0
1,HealthCare.com U65,476-322-7661,U65_HC_SL_G_RO_state-terms-kwurl-experiment_AD...,KS.1.1.10.1.5.00.1.0P.2_NF_NA_Cheap-Health-Plan,10760603187,102867791421,544255696975,+kansas +cheap +plans +for +2018,USD,0,...,--,--,--,0.0,--,0.0,0,--,--,0
2,HealthCare.com U65,476-322-7661,U65_HC_SL_G_RO_state-terms-kwurl-experiment_AD...,NV.1.1.10.1.5.00.1.0e.2_NF_NA_Obamacare-Sign-Up,10760603187,105331381359,482888648805,+2018 +obamacare +sign +up +nevada,USD,0,...,--,--,--,0.0,--,0.0,0,--,--,0
3,HealthCare.com U65,476-322-7661,U65_HC_SL_G_RO_state-terms-kwurl-experiment_AD...,WY.1.1.10.1.5.00.1.2G.2_NF_NA_Cobra,10760603187,105331410439,828823157196,+wy +cobra +plans +2020,USD,0,...,--,--,--,0.0,--,0.0,0,--,--,0
4,HealthCare.com U65,476-322-7661,U65_HC_SL_G_RO_state-terms-kwurl-experiment_AD...,AK.1.1.10.1.5.00.1.0e.2_NF_NA_Obamacare,10760603187,105331377359,377984945895,2018 obamacare alaska plans,USD,0,...,--,--,--,0.0,--,0.0,0,--,--,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1021229,HealthCare.com U65,476-322-7661,U65_HC_SL_G_RO_state-terms-kwurl-experiment_AD...,ND.1.1.10.1.5.00.1.0p.2_NF_NA_Trumpcare,10760603187,102867795741,519149961064,health insurance north dakota state trumpcare,USD,0,...,--,--,--,0.0,--,0.0,0,--,--,0
1021230,HealthCare.com U65,476-322-7661,U65_HC_SL_G_RO_state-terms-kwurl-experiment_AD...,MI.1.1.10.1.5.00.1.0e.2_NF_NA_Obamacare,10760603187,109125933114,829065459031,+2020 +obama +care +in +mi,USD,0,...,--,--,--,0.0,--,0.0,0,--,--,0
1021231,HealthCare.com U65,476-322-7661,U65_HC_SL_G_RO_state-terms-kwurl-experiment_AD...,MI.1.1.10.1.5.00.1.0e.2_NF_NA_Obamacare,10760603187,109125933114,475034504067,health insurance michigan state obamacare,USD,0,...,--,--,--,0.0,--,0.0,0,--,--,0
1021232,HealthCare.com U65,476-322-7661,U65_HC_SL_G_RO_state-terms-kwurl-experiment_AD...,WA.1.1.10.1.5.00.1.0P.2_NF_NA_Cheap-Health-Ins...,10760603187,105331374199,351928822230,cheap health insurance for wa,USD,0,...,--,--,--,0.0,--,0.0,0,--,--,0


In [189]:
data['Account name'].value_counts()

HealthCare.com U65    892005
HealthCare.com O65    129229
Name: Account name, dtype: int64

In [190]:
# data['Ad group'].value_counts()

In [191]:
data['Ad group ID'].nunique()

72429

### T1

In [192]:
data.loc[data['Account name'] == 'HealthCare.com U65']['Ad group ID'].nunique()

53080

In [193]:
data.loc[data['Account name'] == 'HealthCare.com O65']['Ad group ID'].nunique()

19349

In [194]:
data.loc[data['Account name'] == 'HealthCare.com O65']['Campaign ID'].nunique()

29

In [195]:
data.loc[data['Account name'] == 'HealthCare.com U65']['Ad group ID'].nunique() + data.loc[data['Account name'] == 'HealthCare.com O65']['Ad group ID'].nunique()

72429

### T2

In [196]:
data['Ad group ID'].unique()

array([109125946314, 102867791421, 105331381359, ..., 102867810661,
       102867812861, 102867808261])

In [197]:
data['Keyword ID'].unique()

array([668909987272, 544255696975, 482888648805, ..., 829065459031,
       475034504067, 543081263881])

In [198]:
# for i in data['Ad group ID'].unique():

#     print(i, data.loc[data['Ad group ID'] == i, 'Search keyword'])

In [199]:
# data['Ad group ID'].unique()

In [200]:
# print(data.loc[data['Ad group ID'] == 109125946314, 'Search keyword'])

In [201]:
# data.loc[data['Ad group ID'] == 105331410439]['Search keyword'].to_list()

In [202]:
# sk_list = data.loc[data['Ad group ID'] == 109125946314]['Search keyword'].to_list()

In [203]:
# text = '.'.join(sk_list)
# text

In [204]:
def summarizer(text):
    # If the length of the text is greater than 20, take a 10th of the sentences
    if text.count(". ") > 20:
      length = int(round(text.count(". ")/10, 0))
    # Otherwise return five sentences
    else:
      length = 1
    # Remove punctuation
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    # Remove stopwords
    processed_text =[word for word in nopunc.split() if word.lower() not in nltk.corpus.stopwords.words('english')]
    # Create a dictionary to store word frequency
    word_freq = {}
    # Enter each word and its number of occurrences
    for word in processed_text:
      if word not in word_freq:
        word_freq[word] = 1
      else:
        word_freq[word] = word_freq[word] + 1
    # Divide all frequencies by max frequency to give store of (0, 1]
    max_freq = max(word_freq.values())
    for word in word_freq.keys():
      word_freq[word] = (word_freq[word]/max_freq)
    # Create a list of the sentences in the text
    sent_list = nltk.sent_tokenize(text)
    # Create an empty dictionary to store sentence scores
    sent_score = {}
    for sent in sent_list:
      for word in nltk.word_tokenize(sent.lower()):
        if word in word_freq.keys():
          if sent not in sent_score.keys():
            sent_score[sent] = word_freq[word]
          else:
            sent_score[sent] = sent_score[sent] + word_freq[word]
    summary_sents = nlargest(length, sent_score, key = sent_score.get)
    summary = ' '.join(summary_sents)
    return summary     

In [205]:
# sum_text = summarizer(text)
# # sum_text

In [206]:
def sum2(sum_text):
    sum_text = sum_text.replace(".", " ")
    sum_text = sum_text.lower()
    # print(sum_text)
    words = sum_text.split()
    words = [word.strip('.,!;()[]') for word in words]
    words = [word.replace("'s", '') for word in words]
    unique = []
    for word in words:
        if word not in unique:
            unique.append(word)
    
    return ' '.join(unique)      

In [207]:
# sum2(sum_text)

In [208]:
# print(unique)

In [209]:
# words

### Final

In [212]:
#Get input
#
id_type = input(" Select:\nAd Group ID (adi) | Keyword ID (ki) ")
print(id_type)

 Select:
Ad Group ID (adi) | Keyword ID (ki)  adi


adi


In [213]:
if id_type=='ki':
    ki = input("Enter Keyword ID")
    print('ki: ', ki)
    adi = data.loc[data['Keyword ID'] == ki]['Ad group ID']
    print('adi: ',adi[0])
    sk_list = data.loc[data['Ad group ID'] == adi[0]]['Search keyword'].to_list()
    # print('sk_list: \n',sk_list)
    text = '.'.join(sk_list)
    # print('text: \n',text)
    sum_text = summarizer(text)
    # print('sum_text: \n',sum_text)
    # sum2(sum_text)
    print('sum2: \n',sum2(sum_text))
elif id_type=='adi':
    adi = input("Enter Ad Group ID")
    print('adi: ',adi)
    sk_list = data.loc[data['Ad group ID'] == int(adi)]['Search keyword'].to_list()
    # print('sk_list: \n',sk_list)
    text = '.'.join(sk_list)
    # print('text: \n',text)
    sum_text = summarizer(text)
    # print('sum_text: \n',sum_text)
    # sum2(sum_text)
    print('sum2: \n',sum2(sum_text))
else:
    print("Wrong ID type")

Enter Ad Group ID 109125946314


adi:  109125946314
sum2: 
 +wv +exchange +2020 healthcare exchange in west virginia wv state health insurance care plans 2020 exchanges of rates affordable individual +west +virginia +individual +health +insurance +exchanges premiums 2018 off +state benefit 2019 +2018 medical +in


In [171]:
more = input(" Show more ")
if more == 'y':
    print('sum_text: \n',sum_text)
elif more == 'yy':
    print('text: \n',text)
elif more == 'yyy':
    print('sk_list: \n',sk_list)
else:
    print("Done")

 Show more  yyy


sk_list: 
 ['hawaii health insurance exchange rates', ' +hawaii +health +insurance +exchange +rates', 'hawaii health insurance exchange rates']


In [165]:
# data.loc[data['Ad group ID'] == 105331410439]['Search keyword'].to_list()

In [95]:
data['Ad group ID'].unique()

array([109125946314, 102867791421, 105331381359, ..., 102867810661,
       102867812861, 102867808261])

In [96]:
data['Keyword ID'].unique()

array([668909987272, 544255696975, 482888648805, ..., 829065459031,
       475034504067, 543081263881])

In [166]:
# print(data.loc[data['Ad group ID'] == 109125946314, 'Search keyword'])